# Assignment 2: Static Portfolio - Part 1 (Intro & Exploratory)

**Author: Luis Eduardo San Martin**

My purpose with this assignment is to use the `altair` visualization library to explore a few of the inequalities of Lima, the capital of Peru. Each graphic presented seeks to represent a particular topic/dimension where the inequality of Lima is reflected.

## Comment on my data preparation

All my data preparation was conducted on the notebooks from [this folder](https://github.com/luisesanmartin/data-vis-public-policy-course/tree/master/notebooks). The raw data I used is stored [here](https://github.com/luisesanmartin/data-vis-public-policy-course/tree/master/data/raw). This raw data comes from these sources:

* **GIS data of the districts of Peru:** Generated and published by INEI, the National Institute of Statistics of Peru. It was published along with the results of the Poverty Map 2013. It is Available for download in [this link](http://iinei.inei.gob.pe/iinei/srienaho/descarga/DocumentosZIP/2013-129/Informacioncartografica.zip).

* **Data from the National Households Survey (ENAHO):** Also collected by INEI. Specifically, I used the 2018 datasets of the health, labor, dwelling and summary modules; along with the 2014-2017 datasets of the dwelling and summary modules. This data can be downloaded from the [INEI microdata repository](http://iinei.inei.gob.pe/microdatos/).

* **District population data from the 2017 National Census:** Unexplicably, INEI has not yet published the microdata of the last National Census conducted in Peru. Instead, only some ad-hoc data queries are available in this [old-fashioned site](http://censos2017.inei.gob.pe/redatam/). From it, I generated queries to download the districts population for Lima and Callao, the two departments in the Lima Metropolitana area.

* **Average results by school of 2016 National Students Evaluation:** This data is generated by the Ministry of Education of Peru, and it is available only through formal requests of access to information conducted in-person. I gained access to it through a request I did in 2018. The raw file I received for my request is available in my [raw data folder](https://github.com/luisesanmartin/data-vis-public-policy-course/tree/master/data/raw) (and publishing it does not violate any law of the Peruvian public data protection regulation).

Using data from several sources required extensive data wrangling and standardization, conducted exclusively in the `pandas` and `geopandas` libraries. Any curious reader is invited to visit my [personal repository of the course](https://github.com/luisesanmartin/data-vis-public-policy-course) to review it.

Now, let us proceed.

## 0 - Importing the libraries

I will make use of `pandas` and `geopandas` for reading and manipulating my data, and of `altair` to produce my visualizations.

In [1]:
import altair as alt
alt.renderers.enable('html')
import geopandas as gpd
import pandas as pd

I will also set a universal path to read my data from.

In [2]:
path = '../../data/clean/'

## 1 - Introducing my classification of Lima

My first plot serves the purpose of introducing the unfamiliar reader with the city of Lima.

With more than 10 million people and a regulation that forbids the construction of skyscrappers for the risk of earthquakes, Lima is a gigantic city in inhabitants and extension. Bounded between the South Pacific Ocean to the West and the beginning of the Andes mountains to the East, it has a weird shape.

This first plot consists of a map of the city of Lima using an aggregation of districts (the administrative division of Lima Metropolitana) into eight distinct groups defined by me. Though this grouping is ultimately arbitrary, it tries to follow a criteria of geographic proximity and road accessibility intra group. Then, this is my classification of the city:

* North Lima
* South Lima
* East Lima
* Old town
* The port
* High income Lima
* San Juan de Lurigancho
* South beach

From now on, these areas will be called zones.

### And now, the code

I will continue now by importing my cleaned data

In [3]:
zones_file = 'zones_lima.geojson'
lima_zones = gpd.read_file(path + zones_file)

`lima_zones` consists of a geographic dataframe where each row is a zone. It contains the GIS data of each zone and its total population. This dataframe was created with district population data from the 2017 National Census and with GIS data of the districts of Peru produced for the 2013 Poverty Map, both published by INEI. The data preparation for this dataframe can be [reviewed here](https://github.com/luisesanmartin/data-vis-public-policy-course/blob/master/notebooks/1_grouping%20districts%20of%20lima.ipynb).

Now I will generate two columns with the latitude and longitude of the centroids of each zone.

In [4]:
lima_zones['centroid_lat'] = lima_zones['geometry'].centroid.y
lima_zones['centroid_lon'] = lima_zones['geometry'].centroid.x

And before we plot, I will replace the name of the zone "San Juan de Lurigancho" with "San Juan", only for this graph. I do this because, apparently, `altair` [does not support yet a multiline option for text encoding](https://github.com/altair-viz/altair/issues/262) and the name "San Juan de Lurigancho" is too long for my plot.

In [5]:
lima_zones2 = lima_zones.replace('San Juan de Lurigancho', 'San Juan')

Now, we plot. I used [this example](https://altair-viz.github.io/gallery/london_tube.html) as a reference for producing this code.

In [6]:
background = alt.Chart(lima_zones2).mark_geoshape().encode(
    color='population'
).properties(
    width=600,
    height=500,
    title={'text': 'This is Lima',
           'subtitle': ['A plot of the city population in 2017, divided by zones',
                        'Data sources: 2017 National Census and 2013 Poverty Map']}
)
labels = alt.Chart(lima_zones2).mark_text().encode(
    latitude='centroid_lat',
    longitude='centroid_lon',
    text='zone'
)

background + labels

alt.LayerChart(...)

With the clear exception of South Beach, every zone in this classification has a population of at least one million people, approximately. North Lima, is the zone with the most population, with more than 2 million people. Given the difficulties in creating proximity clusters of districts in Lima, I consider that this classification is acceptable and I will continue to use throughout this notebook. South beach, though, will probably be an outlier for the next visualizations and I will have to leave it aside in some cases.

### Data types in the Stevens scale

* **zones:** nominal. Though some of the names I used might imply a geographical (North, South, East) or income (Old Town versus High Income Lima) hierachy, they are actually nominal categories
* **population:** interval

### Encodings according to Mackinlay

* **area:** geographic extension of the zone
* **position:** geographic location of the area with respect of the other areas
* **color saturation:** total population

## 2 - Type of access to water by zone

I chose this as my first indicator to show the inequities of Lima because extending the network of piped water in the city poses a series of tremendous challenges for its great extension, topographic variation,  scarce availability of natural water resources and the corruption of the water provider (a public firm), among others.

### My code

I will start by importing my data. Each row in this data table is a unique combination of year and zone. The preparation of this dataframe is available [here](https://github.com/luisesanmartin/data-vis-public-policy-course/blob/master/notebooks/3_arranging%20the%20ENAHO%20data_access%20to%20water.ipynb). The raw data I used comes from the dwelling and summary modules of the National Household Survey data for the years 2014-2018, published by INEI.

In [7]:
water_file = 'access to water by zones_2014-2018.csv'
access_to_water = pd.read_csv(path + water_file)

The columns in the `access_to_water` dataframe are the proportions of population in a year and zone who access water in a particular way. These are the categories:

1. Piped water inside the house
2. Piped water outside the house, inside the property
3. Tanker truck
4. Public water reservoir
5. Water well
6. River or water stream
7. Other water source

The careful reader will have noticed that the order of these categories is not random. They are sorted by quality of the mean to access water, from higher to lower.

Right now, the dataset is in wide format with respect to the type of access to water. For this visualization, I will first have to transform it from wide to long format. To do this, I will add a name stub to all the access to water columns I have.

In [8]:
stub = 'access to water'
columns = ['1-piped water inside the house',
           '2-piped water outside the house, inside the property',
           '3-tanker truck',
           '4-public water reservoir',
           '5-water well',
           '6-river or water stream',
           '7-other water source']
for col in columns:
    access_to_water = access_to_water.rename(columns = {col: stub + '-' + col})

Now I will reshape the data from wide to long

In [9]:
access_to_water2 = pd.wide_to_long(access_to_water,
                                   stubnames=stub,
                                   i=['year', 'zone'],
                                   j='type of access',
                                   sep='-',
                                   suffix='([\w\s\d,]|-)+').reset_index()

After this, each observation is a unique combination of `year`, `zone` and `type of access`. This is exactly what I need for my plot.

Now I will create my visualization. I used [this example](https://altair-viz.github.io/gallery/stacked_bar_chart.html) as the reference for my code.

In [10]:
alt.Chart(access_to_water2[access_to_water2['year'] == 2018]).mark_bar().encode(
    alt.Y('zone', sort=['High income Lima',
                        'Port',
                        'South Lima',
                        'Old town',
                        'North Lima',
                        'East Lima',
                        'San Juan de Lurigancho',
                        'South beach']),
    alt.X('access to water',
          scale=alt.Scale(domain=(0, 1)),
          axis=alt.Axis(title='Access to water (proportion of population)')),
    color = 'type of access',
    order=alt.Order('type of access',
                    sort='ascending')
).properties(
    width = 590,
    title={'text': 'The types of access to water in Lima',
           'subtitle': ['Type of access to water in Lima in 2018, by zone',
                        'The order of the type of access reflects a quality hierarchy',
                        'Data source: 2018 National Household Survey']}
)

alt.Chart(...)

Except for Sout Beach, there is not a stark contrast between each zone in the proportion of population with access to piped water inside the house. High Income Lima reaches a proportion of more than 0.95, buth the Port, South Lima and the Old Town are also above 0.90. South Beach is clearly an oulier in this, and a reason for that might be the expansion of and middle-class beach houses and informal settlements in this area of the city. In general, there is not a big inequality in this indicator among different zones of Lima.

### Data types

* **type of access to water:** ordinal. A higher order means a higher quality of access to water.
* **zones**: nominal
* **proportion of the population**: ratio

### Encodings

* **color hue:** type of access to water
* **position:** quality hierachy in access to water
* **length:** proportion of the population

## 3 Evolution of the access to piped water inside the house

I will continue by exploring the evolution of the access to piped water inside the house, the highest quality type of access to water. I will keep using the `access to water` dataframe, where every observation is a combination of `year` and `zone`, and the proportion of population that has a certain type of access to water is conveniently stored in separated columns.

First, I will remove the name stub in my variable of interest.

In [11]:
rename = {'access to water-1-piped water inside the house': 'piped water inside the house'}
access_to_water = access_to_water.rename(columns=rename)

Now I can produce my graph. I will leave the zone "South beach" out of this visualization because  it is clearly an outlier in this variable. The code for this graph was based on the examples presented in the class lecture.

In [12]:
alt.Chart(access_to_water[access_to_water['zone']!='South beach']).mark_line().encode(
    alt.X('year:N'),
    alt.Y('piped water inside the house',
          scale=alt.Scale(domain=(0.8, 1)),
          axis=alt.Axis(title='Piped water inside the house (proportion)')),
    color = 'zone'
).properties(
    width = 590,
    title={'text': 'Evolution of the access to piped water in Lima',
           'subtitle': ['Proportion of population with access to piped', 
                        'water inside the house by zone, 2014-2018',
                        'Data sources: 2014-2018 National Household Survey']}
)

alt.Chart(...)

The last five years of available data do not show a clear trend in the increase of access to piped water by zone Lima, though in the last year (2018) all areas increased it. Also importantly, this visualization does not bring evidence of a clear diverging trend by zone that could exacerbate existing inequalities (with the slight exception of South Beach, not included in this graph). All things considered, it seems that access to water is not really very unequal among these zones in Lima.

### Data types

* **zones**: nominal
* **year:** interval
* **proportion of the population**: ratio

### Encodings

* **color hue:** zones
* **connection:** same zone
* **slope:** year-to-year difference in the proportion
* **position:** proportion and year

## 4 - Distribution by zones of the access to formal jobs

Leaving access to water behind, now I will analyze the access to formal jobs. Specifically, I will compare how the whole population of Lima is distributed by zone with how the whole population of *people with formal jobs* is distributed by zone.

The Peruvian government does not have a precise definition of formal jobs, and most approaches usually rely on if a worker is affiliated to a retirement and on if she has some kind of health provision plan. Specifically, my definition of formal job is the following:

* The worker is affiliated to a retirement plan.
* The worker is affiliated to at least one these types of health provision: the general health provider for workers, a private health provider, or the police/military health provider.

The data wranging for the dataframe I will use for this visualization is [here](https://github.com/luisesanmartin/data-vis-public-policy-course/blob/master/notebooks/4_arranging%20the%20ENAHO%20data_formal%20jobs.ipynb). The raw data comes from the health, labor and summary modules of the 2018 National Household Survey, collected and published by INEI.

Opening the data file:

In [13]:
formal_jobs_file = 'percentage formal jobs by zone_lima_2018.csv'
formal_jobs = pd.read_csv(path + formal_jobs_file)

The dataframe `formal_jobs` contains only one row: the year 2018. It has columns with the proportion of population from the different zones of Lima, among the whole population in the city with formal jobs. In other words, this small dataframe is wide with respect to `zone`. Given that I need it in long format, I will first add a stub name to every zone variable and then reshape it.

In [14]:
stub = 'formal jobs'
zones = ['Port',
         'North Lima',
         'Old town',
         'San Juan de Lurigancho',
         'East Lima',
         'High income Lima',
         'South Lima',
         'South beach']
for col in zones:
         formal_jobs = formal_jobs.rename(columns={col: stub + '-' + col})

Reshaping the dataframe to long format:

In [15]:
formal_jobs2 = pd.wide_to_long(formal_jobs,
                               stubnames=stub,
                               i='year',
                               j='zone',
                               sep='-',
                               suffix='[\w\s]+').reset_index()

This result has the proportion of population with formal jobs by zone, but I still do not have the proportion of all the population by zone. Then, I will bring the `population` column from the `lima_zones` dataframe, which I used for my first visualization.

In [16]:
formal_jobs2 = pd.merge(formal_jobs2, lima_zones[['zone', 'population']], how='inner', on='zone')

The result is a dataframe in wide format with respect to `formal jobs` and `population`. To produce the graph I have in mind, I need the dataframe in long format with respect to those two columns, so I will group them in a single columns named `indicator`.

Adding a stub prefix for reshaping:

In [17]:
formal_jobs2 = formal_jobs2.rename(columns={'formal jobs': 'value-population with formal jobs',
                                            'population': 'value-population'})

Reshaping:

In [18]:
formal_jobs2 = pd.wide_to_long(formal_jobs2,
                               stubnames='value',
                               i='zone',
                               j='indicator',
                               suffix='[\w\s]+',
                               sep='-').reset_index()

Finally, the plot. This plot will compare the proportion of people by zone and the proportion of people with formal jobs by zone, in two parallel normalized stacked bars. Please notice that I am leaving out the area "South beach" because its share of the population is so low that it could not be visualized in the graphic with the other zones.

The code for producing this graph was referenced on the examples presented in the lecture.

In [19]:
alt.Chart(formal_jobs2[formal_jobs2['zone']!='South beach']).mark_bar().encode(
    alt.X('indicator'),
    alt.Y('sum(value)',
          stack='normalize',
          axis=alt.Axis(title='Proportion of population'),
          sort='descending'),
    alt.Color('zone')
).properties(
    width = 200,
    title={'text': 'Distribution of formal jobs in Lima',
           'subtitle': ['Comparing the entire population and the ', 
                        'population with formal jobs by zone, in 2018',
                        'Data sources: 2017 National Census and',
                        '2018 National Household Survey']}
)

alt.Chart(...)

Except for the Old Town and North Lima zones, the graph does not show salient differences betwen the proportion of population considered in both indicators. More importantly, it is noticeable that the proportion of population from High Income Lima is lower among the population with formal jobs than among the whole population. This is a completely unexpected result, and possibly an indicator that my classification of High Income Lima is not as high income as I thought, or that my criteria for formal jobs (which I expected to be correlated with wealth) need to be more strict.

### Data types

* **zones**: nominal
* **indicator:** nominal
* **proportion of the population**: ratio

### Encodings

* **color hue:** zones
* **length:** proportion of population
* **position:** indicator

## 5 - Distribution of the quality of public primary schools by area

Lastly, the final inequality I will explore is the quality of public primary schools by zone. To show this, I will use the results of the last National Evaluation of Students as a proxy for school quality. Though I initially considered a Ridgeplot for this, I finally decided for a Stripplot to show the distribution by zone of the average result by school of the National Students Evaluation for the second year of primary, for all public schools in Lima. The National Students Evaluation was a standardized test conducted by the Ministry of Education of Peru to track the progress of the students' learnings year by year. It was discontinued after 2016 and replaced by a sampled evaluation instead of a nationwide one.

The dataframe `schools` was prepared in [this notebook](https://github.com/luisesanmartin/data-vis-public-policy-course/blob/master/notebooks/2_cleaning%20IE%202P%20ECE%2016_Lima.ipynb). As mentoined before, its raw data comes from a request of access to public information I conducted in-person with the Ministry of Education of Peru. Every row in the dataframe is a public primary school in Lima Metropolitana, and the list is complete. The columns it has contain the average result in math for second grade of primary, the zone where the school is, and some additional school identificators (name, school ID and others).

Reading the data:

In [20]:
schools_file = 'ECE_by_school_2p_lima_2016.csv'
schools = pd.read_csv(path + schools_file)

This data allows to produce the graph directly. I relied heavily on this [Stripplot example](https://altair-viz.github.io/gallery/stripplot.html) to generate this code.

In [21]:
alt.Chart(schools, width=30).mark_circle(size=8).encode(
    alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    alt.Y('average_score:Q',
      axis=alt.Axis(title='Average test scores')),
    color=alt.Color('zone:N', legend=None),
    column=alt.Column(
        'zone:N',
        header=alt.Header(
            labelAngle=-90,
            titleOrient='top',
            labelOrient='bottom',
            labelAlign='right',
            labelPadding=3,
        ),
    ),
).transform_calculate(
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).properties(
    title={'text': 'Distribution of public primary schools quality in Lima',
           'subtitle': ['Average results by school in the math section',
                        'of the 2016 National Students Evaluation, for',
                        'second grade of primary. Each dot represents a',
                        'single school',
                        'Data source: Ministry of Education of Peru']}
)

alt.Chart(...)

The graph shows that the number of public schools in South Beach is very low, and that North Lima has a higher number of public schools that all the other zones. Regarding the distribution of results, the graph does not seem to show clear differences by zone. The Port has a higher concentration of schools among the top of the graph, but it is not really a prominent difference.

### Data types

* **average score**: interval
* **zone**: nominal

### Encodings

* **color hue:** zones
* **position:** zone and average score

## Final remarks

In general, I feel that my visualizations failed to my purpose of showing the stark inequalities existing in Lima. Instead of that, they have led to the following hypotheses/ideas:

* Lima is actually much more equal than I expected among the zones I used in the indicators I presented
* My zones grouping is possibly too big to show the clusters of good living standards and underdevelopment present in the city
* Using several data sources is a big constraint in any visualization analysis, as the data standardization takes a significant amount of time and effort
* I should probably consult some bibliography on the issue I want to show to have a clearer idea of the story I want to tell, and the visualizations that will help me on that end